## A melhorar antes de mandar ao professor daniel

F1 score - https://torchmetrics.readthedocs.io/en/stable/classification/f1_score.html

ter resto das capacidades de PyTorch lightning

dividir em deve test ainda para comparar melhor

Documentar tudo melhor

### Ver minhas notas para ver o que mais se pode mudar

https://www.kaggle.com/datasets/fedesoriano/heart-failure-prediction

init.xavier_uniform_(self.lstm.weight)


In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder

import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import pytorch_lightning as pl
import torch.nn.init as init

import torchmetrics

import os

In [2]:
num_cores_os = os.cpu_count()
print(f"Number of available CPUs: {num_cores_os}")

# Check if CUDA (GPU support) is available
if torch.cuda.is_available():
    # Get the number of available GPUs and store it in a variable
    num_gpus = torch.cuda.device_count()
else:
    num_gpus = 0  # No GPUs available

print(f"Number of available GPUs: {num_gpus}")

Number of available CPUs: 8
Number of available GPUs: 0


In [3]:
# Load your dataset, assuming it's in a CSV file
df = pd.read_csv('heart.xls')

print(df)

     Age Sex ChestPainType  RestingBP  Cholesterol  FastingBS RestingECG  \
0     40   M           ATA        140          289          0     Normal   
1     49   F           NAP        160          180          0     Normal   
2     37   M           ATA        130          283          0         ST   
3     48   F           ASY        138          214          0     Normal   
4     54   M           NAP        150          195          0     Normal   
..   ...  ..           ...        ...          ...        ...        ...   
913   45   M            TA        110          264          0     Normal   
914   68   M           ASY        144          193          1     Normal   
915   57   M           ASY        130          131          0     Normal   
916   57   F           ATA        130          236          0        LVH   
917   38   M           NAP        138          175          0     Normal   

     MaxHR ExerciseAngina  Oldpeak ST_Slope  HeartDisease  
0      172              N  

In [4]:
# Encode categorical variables
encoder = LabelEncoder()
categorical_cols = ["Sex", "ChestPainType", "RestingECG", "ExerciseAngina", "ST_Slope"]
for col in categorical_cols:
    df[col] = encoder.fit_transform(df[col])


print(df)

     Age  Sex  ChestPainType  RestingBP  Cholesterol  FastingBS  RestingECG  \
0     40    1              1        140          289          0           1   
1     49    0              2        160          180          0           1   
2     37    1              1        130          283          0           2   
3     48    0              0        138          214          0           1   
4     54    1              2        150          195          0           1   
..   ...  ...            ...        ...          ...        ...         ...   
913   45    1              3        110          264          0           1   
914   68    1              0        144          193          1           1   
915   57    1              0        130          131          0           1   
916   57    0              1        130          236          0           0   
917   38    1              2        138          175          0           1   

     MaxHR  ExerciseAngina  Oldpeak  ST_Slope  Hear

In [5]:
features = df.drop('HeartDisease', axis=1).values
target = df['HeartDisease'].values
print(features)

[[40.   1.   1.  ...  0.   0.   2. ]
 [49.   0.   2.  ...  0.   1.   1. ]
 [37.   1.   1.  ...  0.   0.   2. ]
 ...
 [57.   1.   0.  ...  1.   1.2  1. ]
 [57.   0.   1.  ...  0.   0.   1. ]
 [38.   1.   2.  ...  0.   0.   2. ]]


# Normalize the dataset

In [6]:
scaler = StandardScaler()

features = scaler.fit_transform(features)

print(features)

[[-1.4331398   0.51595242  0.22903206 ... -0.8235563  -0.83243239
   1.05211381]
 [-0.47848359 -1.93816322  1.27505906 ... -0.8235563   0.10566353
  -0.59607813]
 [-1.75135854  0.51595242  0.22903206 ... -0.8235563  -0.83243239
   1.05211381]
 ...
 [ 0.37009972  0.51595242 -0.81699495 ...  1.21424608  0.29328271
  -0.59607813]
 [ 0.37009972 -1.93816322  0.22903206 ... -0.8235563  -0.83243239
  -0.59607813]
 [-1.64528563  0.51595242  1.27505906 ... -0.8235563  -0.83243239
   1.05211381]]


In [7]:
# Convert features and target to NumPy arrays
X = np.array(features, dtype=np.float32)
y = np.array(target, dtype=np.float32)

In [8]:
# Split the dataset into training and devidation sets
X_train, X_dev_and_test, y_train, y_dev_and_test = train_test_split(X, y, test_size=0.4, random_state=42)

X_dev, X_test, y_dev, y_test = train_test_split(X_dev_and_test, y_dev_and_test, test_size=0.5, random_state=42)

In [9]:
#Hyperparameters
input_dim=11
hidden_dim=64
num_layers=2
output_dim=1

current_batch_size=64
num_classes=2

features = 10

# LSTM models require input data to be three-dimensional, with the dimensions representing (batch size, time_steps, and features) - ver como preparar informação para isto

    The input data is a 3D tensor, where the first dimension represents the batch size, the second dimension represents the sequence length, and the third dimension represents the features at each time step.

    
    The target data can be a 2D tensor, where the first dimension represents the batch size, and the second dimension represents the number of classes for classification. In binary classification, it's often a 2D tensor with shape (batch_size, 2) or a 1D tensor with shape (batch_size,), where each value represents the binary label (0 or 1).

In [19]:
# Convert NumPy arrays to PyTorch tensors
X_train = torch.tensor(X_train).clone().detach()
y_train = torch.tensor(y_train).clone().detach()

X_dev = torch.tensor(X_dev).clone().detach()
y_dev = torch.tensor(y_dev).clone().detach()


X_test = torch.tensor(X_test).clone().detach()
y_test = torch.tensor(y_test).clone().detach()

print(X_train.shape)
print(y_train.shape)

print(X_dev.shape)
print(y_dev.shape)

print(X_test.shape)
print(y_test.shape)



torch.Size([550, 11])
torch.Size([550])
torch.Size([184, 11])
torch.Size([184])
torch.Size([184, 11])
torch.Size([184])


/tmp/ipykernel_255180/2531228077.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_train = torch.tensor(X_train).clone().detach()
/tmp/ipykernel_255180/2531228077.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_train = torch.tensor(y_train).clone().detach()
/tmp/ipykernel_255180/2531228077.py:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X_dev = torch.tensor(X_dev).clone().detach()
/tmp/ipykernel_255180/2531228077.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sou

In [17]:
class HeartPredictionModel(pl.LightningModule):
    def __init__(self, input_dim, hidden_dim, num_layers, output_dim):
        super(HeartPredictionModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, bidirectional=True)
        #init.xavier_uniform_(self.lstm.weight)
        self.fc = nn.Linear(2* current_batch_size*hidden_dim, output_dim) 
        init.xavier_uniform_(self.fc.weight)
        
        
    def forward(self, x):
        out, _ = self.lstm(x)
        out = torch.flatten(out)  # Get the last output from the sequence
        out = self.fc(out)
        return out 

    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.001)
        scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)
        return [optimizer], [scheduler]
        
    def training_step(self, batch, batch_idx):
        inputs, targets = batch
        current_batch_size = inputs.shape[0]
        outputs = self(inputs)
        loss = nn.BCELoss()(outputs, targets)
        self.log('train_loss', loss)
        return loss

    def validation_step(self, batch, batch_idx):
        inputs, targets = batch
        outputs = self(inputs)
        loss = nn.BCELoss()(outputs, targets)
        self.log('val_loss', loss)
        return loss

    def train_dataloader(self):
        dataset = TensorDataset(X_train, y_train)
        return DataLoader(dataset, batch_size=current_batch_size, shuffle=True, drop_last=True, num_workers=num_cores_os)

    def val_dataloader(self):
        dataset = TensorDataset(X_dev, y_dev)
        return DataLoader(dataset, batch_size=current_batch_size, drop_last=True, num_workers=num_cores_os)

In [ ]:
model = HeartPredictionModel(input_dim=input_dim, hidden_dim=hidden_dim, num_layers=num_layers, output_dim=output_dim)

# Initialize a PyTorch Lightning trainer
trainer = pl.Trainer(max_epochs=10)  # Set the number of epochs and GPU usage as needed

# Train the model
trainer.fit(model)